In [1]:
from transformers import UperNetForSemanticSegmentation, AutoImageProcessor
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from evaluate import load

/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_dir = '/home/a-ploskin/repos/TerraLabel/data/task_0/data'
masks_dir = '/home/a-ploskin/repos/TerraLabel/data/masks'

In [3]:
class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, image_processor, train, img_dir=img_dir, masks_dir=masks_dir):
        

        self.img_pathes = [ f'{img_dir}/{item}' for item in sorted(os.listdir(img_dir)) ]
        self.img_pathes = [item for item in self.img_pathes if item[-3:] == 'jpg']
        self.mask_pathes = [ f'{masks_dir}/{item}' for item in sorted(os.listdir(masks_dir)) ]

        if train == False: 
            self.img_pathes = self.img_pathes[:20]
            self.mask_pathes = self.mask_pathes[:20]
        
        self.image_processor = image_processor

        assert len(self.img_pathes) == len(self.mask_pathes), "There must be as many images as there are segmentation maps"

    def __len__(self):
        return len(self.img_pathes)

    def __getitem__(self, idx):

        mask = np.load(self.mask_pathes[idx])
        image = np.array( Image.open(self.img_pathes[idx])) 

        encoded_inputs = self.image_processor(image, mask, return_tensors="pt")

        for k,v in encoded_inputs.items():
            encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs
     

In [4]:
model = UperNetForSemanticSegmentation.from_pretrained("openmmlab/upernet-swin-large")
processor = AutoImageProcessor.from_pretrained("openmmlab/upernet-swin-large")

model.to('cuda')

UperNetForSemanticSegmentation(
  (backbone): SwinBackbone(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=192, out_features=192, bias=True)
                  (key): Linear(in_features=192, out_features=192, bias=True)
                  (value): Linear(in_features=192, out_features=192, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): Swin

In [5]:
train_ds = SemanticSegmentationDataset(processor, train=True)
eval_ds = SemanticSegmentationDataset(processor, train=False)

In [6]:
train_dataloader = DataLoader(train_ds, batch_size=2, shuffle=True, drop_last=True)
eval_dataloader = DataLoader(eval_ds, batch_size=2, shuffle=True, drop_last=True)

In [7]:
log_path = './logs'
writer = SummaryWriter(log_path)
metric = load("mean_iou")

In [8]:
import torch
from tqdm import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
device = torch.device("cuda:0")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loss_iter = []
train_loss_epoch = []
eval_iou = []
eval_acc = []
eval_loss = []
model.train()

for epoch in tqdm(range(200)):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    curr_epoch_loss = []
    curr_epoch_eval_loss = []
    for idx, batch in enumerate(train_dataloader):
        # print(idx)
        
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)
        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()
        #print()
        train_loss_iter.append(loss.item())
        curr_epoch_loss.append(loss.item())
        writer.add_scalar("Train/loss_step", train_loss_iter[-1], idx + epoch * len(train_dataloader))
        writer.add_scalar("Train/epoch", epoch + 1, idx + epoch * len(train_dataloader))
    train_loss_epoch.append(sum(curr_epoch_loss) / len(curr_epoch_loss))   
    writer.add_scalar("Train/loss_epoch", train_loss_epoch[-1], epoch + 1)

    with torch.no_grad():
        for batch in eval_dataloader:
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss, logits = outputs.loss, outputs.logits
            curr_epoch_eval_loss.append(outputs.loss.item())
            upsampled_logits = torch.nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
            predicted = upsampled_logits.argmax(dim=1)
            metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())
            
        metrics = metric.compute(num_labels=6, 
                                       ignore_index=255,
                                       reduce_labels=False, # we've already reduced the labels before)
            )
        eval_iou.append( metrics["mean_iou"])
        eval_acc.append(metrics["mean_accuracy"])
        eval_loss.append(sum(curr_epoch_eval_loss) / len(eval_dataloader))
        
        writer.add_scalar("Eval/loss",eval_loss[-1], epoch + 1)
        writer.add_scalar("Eval/Accuracy", metrics["mean_accuracy"], epoch + 1)
        writer.add_scalar("Eval/IoU", metrics["mean_iou"], epoch + 1)
        
        print("Mean_iou:", metrics["mean_iou"])
        print("Loss:", train_loss_epoch[-1])
        print("Mean accuracy:", metrics["mean_accuracy"])

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0


/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
  0%|          | 1/200 [00:56<3:08:43, 56.90s/it]

Mean_iou: 0.33513295392727
Loss: 5.279483486966389
Mean accuracy: 0.48540135074217655
Epoch: 1


  1%|          | 2/200 [01:53<3:07:02, 56.68s/it]

...

Mean_iou: 0.7926909418979502
Loss: 0.7534447294182893
Mean accuracy: 0.865568251787816
Epoch: 15


  8%|▊         | 16/200 [15:08<2:54:16, 56.83s/it]

Mean_iou: 0.8025793157789013
Loss: 0.6807066912694675
Mean accuracy: 0.8616639047594945
Epoch: 16


  8%|▊         | 16/200 [15:40<3:00:20, 58.81s/it]


KeyboardInterrupt: 